In [2]:
#2.
# H0: Brak wpływu jakiegokolwiek składnika na czas reakcji, H1: Chociaż jeden składnik wpływa na czas reakcji
partia <- rep(c("P1", "P2", "P3", "P4", "P5"), 5)
dzien <- c(rep("D1",5), rep("D2",5), rep("D3",5), rep("D4",5), rep("D5",5))
skladnik <- c("A","C","B","D","E",
             "B","E","A","C","D",
             "D","A","C","E","B",
             "C","D","E","B","A",
             "E","B","D","A","C")
efekt <-      c(8, 11,4, 6, 4,
                7, 2, 9, 8, 2,
                1, 7,10, 6, 3,
                7, 3, 1, 6, 8,
                3, 8, 5,10, 8)

x = matrix(c(efekt), nrow=5, ncol=5, byrow=FALSE)
colnames(x) <- c("D1", "D2", "D3", "D4", "D5") 
rownames(x) <- c("P1", "P2", "P3", "P4", "P5")
lista_danych <- data.frame(dzien, partia, skladnik, matrix(x))

In [3]:
analiza_wariancji <- function(x, estymacja=FALSE, a.brak=0, b.brak=0){
  N = length(x)
  p = nrow(x) #p=5

  lista_danych <- data.frame(dzien, partia, skladnik, matrix(x))
  yi. <- rowSums(x, na.rm=TRUE)
  y.j <- aggregate(. ~ skladnik, data=lista_danych, sum)[4] # y.j.
  y..k <- colSums(x, na.rm=TRUE) 
  y.. <- sum(x, na.rm=TRUE)
  
  if (estymacja==TRUE){
    nr_elementu <- matrix(c(skladnik), nrow=nrow(x), ncol=ncol(x))[a.brak, b.brak]
    nr_elementu <- match(nr_elementu, LETTERS[1:26]) #u nas=5
    x_est <- (p*(yi.[a.brak]+ y.j[nr_elementu,] + y..k[b.brak])  - (2*y..)) / ((p-2)*(p-1))
    H <- (y.. - yi.[a.brak] - y..k[b.brak] - (p-1)*y.j[nr_elementu,])**2 / (((p-2)**2)*(p-1)**2)
    x[a.brak, b.brak] = x_est #Dodana wyestymowana brakująca
    lista_danych <- data.frame(dzien, partia, skladnik, matrix(x))
    yi. <- rowSums(x, na.rm=TRUE)
    y.j <- aggregate(. ~ skladnik, data=lista_danych, sum)[4] # y.j.
    y..k <- colSums(x, na.rm=TRUE) 
    y.. <- sum(x, na.rm=TRUE) 
    SSA = sum(y.j**2)/p -(y..**2/N) - H }
    
  else{ SSA = sum(y.j**2)/p -(y..**2/N) }
  SST = sum(matrix(x)**2)-(y..**2/N)
  SSC = sum(y..k**2)/p - (y..**2/N)
  SSR  = sum(yi.**2)/p - (y..**2/N)
  SSE = SST - SSA - SSC - SSR
  print(SSE)
  
  #Stopnie swobody:
  dfA = dfR = dfC = p-1
  
  if (estymacja==TRUE){ 
          dfE = (p-2)*(p-1)-1
          dfT = p**2 - 1 -1 }
  else{   dfE = (p-2)*(p-1)
          dfT = p**2 - 1  }

  #Średnie kwadraty:
  MSA <- SSA/dfA
  MSR <- SSR/dfR
  MSC <- SSC/dfC
  MSE <- SSE/dfE
  #F statistics:
  
  F_case = MSA/MSE
  tabela <- rbind(c(SSA, dfA, MSA, F_case),
                  c(SSR, dfR, MSR, "-"),
                  c(SSC, dfC, MSC, "-"),
                  c(SSE, dfE, MSE, "-"),
                  c(SST, dfT, "-", "-"))
  colnames(tabela) <- c("SumaKwadratow", "StopnieSwobody", "SredniKwadrat", "F")
  rownames(tabela) <- c("obiekty", "wiersze", "kolumny", "blad", "calkowita")
return (tabela) }

analiza_wariancji(x)

f0.95 <- qf(0.95, p-1, (p-2)*(p-1))
f0.95 # 3.259 #Nasze 11.309 > 3,259 więc nalezy odrzucic hipotezę zerową

[1] 37.52


,SumaKwadratow,StopnieSwobody,SredniKwadrat,F
obiekty,141.44,4,35.36,11.3091684434968
wiersze,15.4399999999999,4,3.85999999999999,-
kolumny,12.24,4,3.06,-
blad,37.5200000000001,12,3.12666666666667,-
calkowita,206.64,24,-,-


ERROR: Error in qf(0.95, p - 1, (p - 2) * (p - 1)): nie znaleziono obiektu 'p'


In [4]:
#b)
x2 <- x
x2[3,4] <- NA
analiza_wariancji(x2, a.brak=3, b.brak=4, estymacja=TRUE)
#F=13.45

      P3 
42.34444 


,SumaKwadratow,StopnieSwobody,SredniKwadrat,F
obiekty,120.633333333333,4,30.1583333333333,7.83436106008913
wiersze,16.0944444444443,4,4.02361111111108,-
kolumny,8.76111111111095,4,2.19027777777774,-
blad,42.3444444444448,11,3.84949494949498,-
calkowita,187.833333333333,23,-,-


In [5]:
#c) Komponenty modelu
mu = mean(x)
alfa = rowMeans(x) - mu #efekt i wiersza
tj = aggregate(. ~ skladnik, data=lista_danych, mean)[4] #efekt jtego obiektu
beta = colMeans(x) - mu #efekt ktej kolumny
cat(paste("Model: y=", mu, "+"))
alfa
tj
beta

Model: y= 5.88 +

P1    P2    P3    P4    P5 
-0.68  0.32 -0.08  1.32 -0.88

matrix.x.
8.4
5.6
8.8
3.4
3.2


D1    D2    D3    D4    D5 
 0.72 -0.28 -0.48 -0.88  0.92

In [6]:
#d) quantile(alfa, p_v/2,1-(p_v/2))
# quantile(tj, p_v/2,1-(p_v/2))
# quantile(beta, p_v/2,1-(p_v/2))
dane = cbind(as.list(skladnik), as.list(x))
dane = data.frame(dane)
colnames(dane) = c("Lac", "Nr")
SSE =  37.52
y.x. = c(mean(unlist(subset(dane, skladnik=="A")$Nr)), mean(unlist(subset(dane, skladnik=="B")$Nr)), mean(unlist(subset(dane, skladnik=="C")$Nr)), mean(unlist(subset(dane, skladnik=="D")$Nr)), mean(unlist(subset(dane, Lac=="E")$Nr)))

alfa = 0.05
p=5
MSE <- SSE/((p-2)*(p-1))
N = p^2
a = nrow(x)
for(i in 1:(length(y.x.)-1)){
  for(j in (i+1):length(y.x.)){
    print(c(y.x.[i]-y.x.[j],      y.x.[i]-y.x.[j]-qt(alfa/2,(N-1))*sqrt(2*MSE/N),    y.x.[i]-y.x.[j]+qt(alfa/2,(p-1))*sqrt(2*MSE/N)))  } }

[1] 2.800000 3.832224 1.411407
[1] -0.4000000  0.6322244 -1.7885927
[1] 5.000000 6.032224 3.611407
[1] 5.200000 6.232224 3.811407
[1] -3.200000 -2.167776 -4.588593
[1] 2.2000000 3.2322244 0.8114073
[1] 2.400000 3.432224 1.011407
[1] 5.400000 6.432224 4.011407
[1] 5.600000 6.632224 4.211407
[1]  0.200000  1.232224 -1.188593


In [13]:
#3)
montaz <- rep(c("M1", "M2", "M3", "M4"), 4)
operator <- c(rep("O1",4), rep("O2",4), rep("O3",4), rep("O4",4))
metoda <- c("C", "D", "A", "B",
              "B", "C", "D", "A",
              "A", "B", "C", "D",
              "D", "A", "B", "C")
czas <-       c(10, 14, 7, 8,
                7, 18, 11, 8,
                5, 10, 11, 9,
                10, 10, 12, 14)


x = matrix(c(czas), nrow=4, ncol=4, byrow = TRUE)

analiza_wariancji <- function(x, estymacja=FALSE, a.brak=0, b.brak=0){
  N = length(x)
  p = nrow(x) #p=4
  
  lista_danych <- data.frame(montaz, operator, metoda, matrix(x))
  yi. <- rowSums(x, na.rm=TRUE)
  y.j <- aggregate(. ~ metoda, data=lista_danych, sum)[4] # y.j.
  y..k <- colSums(x, na.rm=TRUE) 
  y.. <- sum(x, na.rm=TRUE)
  
  if (estymacja==TRUE){
    nr_elementu <- matrix(c(metoda), nrow=nrow(x), ncol=ncol(x))[a.brak, b.brak]
    nr_elementu <- match(nr_elementu, LETTERS[1:26]) #u nas=5
    x_est <- (p*(yi.[a.brak]+ y.j[nr_elementu,] + y..k[b.brak])  - (2*y..)) / ((p-2)*(p-1))
    H <- (y.. - yi.[a.brak] - y..k[b.brak] - (p-1)*y.j[nr_elementu,])**2 / (((p-2)**2)*(p-1)**2)
    x[a.brak, b.brak] = x_est #Dodana wyestymowana brakująca
    lista_danych <- data.frame(dzien, partia, skladnik, matrix(x))
    yi. <- rowSums(x, na.rm=TRUE)
    y.j <- aggregate(. ~ skladnik, data=lista_danych, sum)[4] # y.j.
    y..k <- colSums(x, na.rm=TRUE) 
    y.. <- sum(x, na.rm=TRUE) }
  
  #Sumy kwadratów
  if (estymacja==TRUE){
        SST = sum(matrix(x)**2)-(y..**2/N)  - H }
  else{ SST = sum(matrix(x)**2)-(y..**2/N)}
  SSA = sum(y.j**2)/p -(y..**2/N)
  SSC = sum(y..k**2)/p - (y..**2/N)
  SSR  = sum(yi.**2)/p - (y..**2/N)
  
  SSE = SST - SSA - SSC - SSR
  print(SSE)
  
  #Stopnie swobody:
  dfA = dfR = dfC = p-1
  
  if (estymacja==TRUE){ 
          dfE = (p-2)*(p-1)-1
          dfT = p**2 - 1 -1 }
  else{   dfE = (p-2)*(p-1)
          dfT = p**2 - 1  }
  
  
  #Średnie kwadraty:
  MSA <- SSA/dfA
  MSR <- SSR/dfR
  MSC <- SSC/dfC
  MSE <- SSE/dfE
  #F statistics:
  
  F_case = MSA/MSE
  tabela <- rbind(c(SSA, dfA, MSA, F_case),
                  c(SSR, dfR, MSR, "-"),
                  c(SSC, dfC, MSC, "-"),
                  c(SSE, dfE, MSE, "-"),
                  c(SST, dfT, "-", "-"))
  colnames(tabela) <- c("SumaKwadratow", "StopnieSwobody", "SredniKwadrat", "F")
  rownames(tabela) <- c("obiekty", "wiersze", "kolumny", "blad", "calkowita")
  return (tabela) }


analiza_wariancji(x)
p = 4
f0 <- qf(0.95, p-1, (p-2)*(p-1))
#Nasze F 13,809 > 4,75. Przyjmujemy hipotezę alternatywną.

[1] 10.5


,SumaKwadratow,StopnieSwobody,SredniKwadrat,F
obiekty,72.5,3,24.1666666666667,13.8095238095238
wiersze,18.5,3,6.16666666666667,-
kolumny,51.5,3,17.1666666666667,-
blad,10.5,6,1.75,-
calkowita,153,15,-,-


In [20]:
#Zad4) łACIŃSKI
montaz <- rep(c("M1", "M2", "M3", "M4"), 4)
operator <- c(rep("O1",4), rep("O2",4), rep("O3",4), rep("O4",4))
metoda <- c("C", "B", "D", "A",
            "B", "C", "A", "D",
            "A", "D", "B", "C",
            "D", "A", "C", "B")
stanowisko <-  c("b","a","d","g",  
                 "g","d","a","b",   
                 "d","g","b","a",  
                 "a","b","g","d")
czas <-       c(11, 10, 14, 8,
                8, 12, 10, 12,
                9, 11, 7, 15,
                9, 8, 18, 6)

x = matrix(c(czas), nrow=4, ncol=4, byrow = TRUE)

analiza_wariancji <- function(x, estymacja=FALSE, a.brak=0, b.brak=0){
  N = length(x)
  p = nrow(x) #p=4
  
  lista_danych <- data.frame(montaz, operator, metoda, stanowisko, matrix(x))
  yi.  <- rowSums(x, na.rm=TRUE)
  y.j <- aggregate(. ~ metoda, data=lista_danych, sum)[5] # y.j.
  y.l =  aggregate(. ~ stanowisko, data=lista_danych, sum)[5] 
  y..k <- colSums(x, na.rm=TRUE) 
  y.. <- sum(x, na.rm=TRUE)
  
  #Sumy kwadratów
  SST = sum(matrix(x)**2)-(y.. **2/N) 
  SSC = sum(y..k**2)/p - (y.. **2/N)
  SSL = sum(y.j**2)/p -(y.. **2/N)
  SSG <- sum(y.l^2)/p - y..**2/N 
  SSR  = sum(yi.**2)/p - (y.. **2/N)
  SSE = SST - SSL - SSG - SSC
  print(SSE)
  
  #Stopnie swobody:
  dfL = dfR = dfC = dfG = p-1
  
  dfE = (p-3)*(p-1)
  dfT = p**2-1
  
  #Średnie kwadraty:
  MSA <- SSL/dfL
  MSG <- SSG/dfG
  MSR <- SSR/dfR
  MSC <- SSC/dfC
  MSE <- SSE/dfE
  #F statistics:
  F_case = MSA/MSE
  tabela <- rbind(c(SSL, dfL, MSA, F_case),
                  c(SSG, dfG, MSG, NA),
                  c(SSR, dfR, MSR, NA),
                  c(SSC, dfC, MSC, NA),
                  c(SSE, dfE, MSE, NA),
                  c(SST, dfT, NA, NA))
  colnames(tabela) <- c("SumaKwadratow", "StopnieSwobody", "SredniKwadrat", "F")
  rownames(tabela) <- c("Łac", "Grec", "wiersze", "kolumny", "blad", "calkowita")
  return (tabela) }

analiza_wariancji(x)  #0,.2678571
f0 <- qf(0.95, p-1, (p-2)*(p-1)) #4.757063
#Stanowisko nie stanowi źródła zmienności, przyjmujemy hipotezę zerową.

[1] 28


,SumaKwadratow,StopnieSwobody,SredniKwadrat,F
Łac,95.5,3,31.8333333,3.410714
Grec,7.5,3,2.5000000,NA
wiersze,0.5,3,0.1666667,NA
kolumny,19.0,3,6.3333333,NA
blad,28.0,3,9.3333333,NA
calkowita,150.0,15,NA,NA
